In [1]:
!pip install transformers

# 1) 데이터 전처리


In [2]:
 import os
 import json
 import numpy as np
 from tqdm import tqdm
 from pathlib import Path
 from transformers import BertTokenizerFast
 import tensorflow as tf
 !wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
 !wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2025-06-25 08:49:46--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   192MB/s    in 0.2s    

2025-06-25 08:49:47 (192 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-06-25 08:49:47--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [3]:
def read_squad(path):
  path = Path(path)
  with open(path, 'rb') as f:
    squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad("KorQuAD_v1.0_train.json")
val_contexts, val_questions, val_answers = read_squad("KorQuAD_v1.0_dev.json")

In [4]:
print("훈련 데이터의 본문 개수:", len(train_contexts))
print("훈련 데이터의 질문 개수:", len(train_questions))
print("훈련 데이터의 답변 개수:", len(train_answers))
print("테스트 데이터의 본문 개수:", len(val_contexts))
print("테스트 데이터의 질문 개수:", len(val_questions))
print("테스트 데이터의 답변 개수:", len(val_answers))

훈련 데이터의 본문 개수: 60407
훈련 데이터의 질문 개수: 60407
훈련 데이터의 답변 개수: 60407
테스트 데이터의 본문 개수: 5774
테스트 데이터의 질문 개수: 5774
테스트 데이터의 답변 개수: 5774


In [5]:
print("첫번째 샘플의 본문", train_contexts[0])

첫번째 샘플의 본문 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [6]:
print("첫번째 샘플의 질문", train_questions[0])

첫번째 샘플의 질문 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [7]:
print("첫번째 샘플의 답변", train_answers[0])

첫번째 샘플의 답변 {'text': '교향곡', 'answer_start': 54}


In [8]:
#train_answers에 종료 인덱스 추가 => 시작 인덱스에 정답 텍스트 길이 더함
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()

        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text, "end_idx 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [9]:
print("첫번째 샘플의 답변:", train_answers[0])

첫번째 샘플의 답변: {'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [10]:
train_contexts[0][54:57]

'교향곡'

In [11]:
#토크나이저, 본문과 질문을 토크나이저 입력으로
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, max_length = 256, truncation = True, padding = 'max_length')
val_encodings = tokenizer(val_contexts, val_questions, max_length = 256, truncation = True, padding = 'max_length')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
print('첫번째샘플의토큰화결과:', train_encodings[0].tokens)

첫번째샘플의토큰화결과: ['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '.', '

In [13]:
print(len(train_encodings[0].tokens))

256


In [14]:
 print('첫 번째 샘플의 정수 인코딩 :', train_encodings[0].ids)

첫 번째 샘플의 정수 인코딩 : [2, 13934, 2236, 2440, 27982, 2259, 21310, 2079, 11994, 3791, 2069, 3790, 1508, 2088, 636, 3800, 2170, 3717, 2052, 9001, 8345, 4642, 2200, 3689, 3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18, 1504, 4342, 27982, 2259, 13934, 2196, 2440, 2170, 1195, 23260, 6233, 17370, 2113, 2165, 2069, 809, 1, 3706, 2052, 2181, 8642, 2145, 7334, 2170, 4983, 2371, 4007, 1065, 5917, 2386, 2559, 4443, 2138, 4026, 2259, 11994, 3791, 2079, 15864, 2170, 5487, 2371, 4683, 3605, 18, 3819, 5986, 27135, 1376, 2645, 2203, 2292, 2079, 5872, 2200, 5986, 4152, 2252, 22835, 16706, 2052, 5485, 2205, 2259, 17087, 2079, 19282, 29, 2517, 2069, 881, 2088, 652, 2073, 23404, 2069, 1122, 2886, 13964, 16, 3982, 2052, 9944, 21, 2429, 2170, 11994, 3791, 2079, 1258, 2465, 6233, 24294, 1504, 3967, 2170, 4027, 2052, 5121, 3979, 2069, 18274, 21575, 23548, 575, 2073, 5292, 2085, 7251, 2116, 1415, 2062, 18, 3776, 2079, 942, 2286, 2446, 4196, 2079, 3640, 6509, 636, 2079, 4450, 2170, 10329, 

In [15]:
 print('첫 번째 샘플의 어텐션 마스크 :', train_encodings[0].attention_mask)

첫 번째 샘플의 어텐션 마스크 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
def add_token_positions(encodings, answers):
    start_positions, end_positions, deleting_list = [], [], []
    for i in tqdm(range(len(answers))):
        #토큰화 전 문자의 인덱스(start, end)로 토큰화 후 문자의 인덱스(start, end) 찾아냄
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] -1))

        #시작 인덱스가 비정상인 경우 -> 본문에 정답이 없는 경우
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        #종료 인덱스가 비정상인 경우 -> 본문에 정답이 없는 경우
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)


    encodings.update({'start_positions' : start_positions, 'end_positions' : end_positions})
    return deleting_list



In [17]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_val = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 410493.95it/s]


In [18]:
print('삭제예정인훈련샘플:', deleting_list_for_train)
print('삭제예정인테스트샘플:',deleting_list_for_val)

삭제예정인훈련샘플: [7, 7, 41, 41, 80, 80, 82, 82, 84, 84, 85, 85, 91, 91, 112, 112, 113, 113, 116, 116, 146, 146, 159, 159, 186, 186, 211, 211, 224, 224, 225, 225, 232, 232, 239, 239, 241, 241, 312, 312, 315, 315, 317, 317, 357, 357, 362, 362, 366, 366, 369, 369, 373, 373, 375, 375, 376, 376, 383, 383, 393, 393, 397, 397, 441, 441, 460, 460, 463, 463, 485, 485, 493, 493, 497, 497, 498, 498, 504, 504, 538, 538, 632, 632, 635, 635, 652, 652, 653, 653, 655, 655, 688, 688, 690, 690, 710, 710, 711, 711, 722, 722, 726, 726, 727, 727, 728, 728, 729, 729, 731, 731, 742, 742, 745, 745, 746, 746, 748, 748, 749, 749, 750, 750, 757, 757, 758, 758, 761, 761, 765, 765, 767, 767, 768, 768, 784, 784, 785, 785, 786, 786, 796, 796, 799, 799, 805, 805, 822, 822, 823, 823, 824, 824, 825, 825, 826, 826, 828, 828, 829, 829, 833, 833, 839, 839, 840, 840, 842, 842, 844, 844, 846, 846, 849, 849, 872, 872, 895, 895, 918, 918, 919, 919, 921, 921, 922, 922, 926, 973, 973, 990, 1013, 1013, 1014, 1014, 1017, 1020, 1020, 10

In [19]:
print('761번샘플의질문:',train_questions[761])
print('-'*50)
print('761번샘플의기존원문:',train_contexts[761])

761번샘플의질문: 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은?
--------------------------------------------------
761번샘플의기존원문: 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름(21)은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영(25-강원도청)이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영(24-전북도청)은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트(네덜란드)가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바(체코-4분01초95)와 3위 올가 그라프(러시아-4분03초47)에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트(1분54초09)에게 돌아갔고 동메달은 하를로터 판바이크(1분54초54)가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라(1분56초40)가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영(25-강원도청)은 2분01초07로 29위

In [20]:
print('761번샘플의기존정답:', train_answers[761])

761번샘플의기존정답: {'text': '8위', 'answer_start': 1050, 'answer_end': 1052}


In [21]:
print('761번샘플전처리후:', tokenizer.decode(train_encodings['input_ids'][761]))

761번샘플전처리후: [CLS] 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름 ( 21 ) 은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영 ( 25 - 강원도청 ) 이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영 ( 24 - 전북도청 [SEP] 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은? [SEP]


In [22]:
def delete_samples(encodings, deleting_list):
    input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis = 0)
    attention_masks =np.delete(np.array(encodings['attention_mask']), deleting_list, axis = 0)
    start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis = 0)
    end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis = 0)

    X_data = [input_ids, attention_masks]
    y_data = [start_positions, end_positions]
    return X_data, y_data

In [23]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_val)

In [24]:
print('-------------삭제전-------------')
print('훈련데이터의샘플의개수:', len(train_contexts))
print('테스트데이터의샘플의개수:', len(val_contexts))
print('-------------삭제후-------------')
print('훈련데이터의샘플의개수:', len(X_train[0]))
print('테스트데이터의샘플의개수:', len(X_test[0]))

-------------삭제전-------------
훈련데이터의샘플의개수: 60407
테스트데이터의샘플의개수: 5774
-------------삭제후-------------
훈련데이터의샘플의개수: 54565
테스트데이터의샘플의개수: 5215


# 2) BERT를 이용한 Question Answering



*   기계독해를 위해 BERT는 시작 인덱스와 종료 인덱스 출력층 두 개 사용
*   입력 길이 512라면, 512개 후보중에서 시작 인덱스, 종료 인덱스를 예측하는 다중 클래스 분류 두 문제를 풀게 됨


In [25]:
from transformers import TFBertModel

class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
        self.qa_outputs = tf.keras.layers.Dense(2, kernel_initializer = tf.keras.initializers.TruncatedNormal(0.02))
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask = attention_mask)

        sequence_output = outputs[0]
        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis = -1)
        start_logits  = tf.squeeze(start_logits, axis = -1)
        end_logits = tf.squeeze(end_logits, axis = -1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [26]:
model = TFBertForQuestionAnswering('klue/bert-base')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=[loss, loss])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [27]:
history = model.fit(X_train, y_train, epochs = 3, verbose = 1, batch_size = 8)

Epoch 1/3


6821/6821 [==============================] - 3246s 469ms/step - loss: 1.1860 - output_1_loss: 0.5353 - output_2_loss: 0.6507
Epoch 2/3
6821/6821 [==============================] - 3193s 468ms/step - loss: 0.7078 - output_1_loss: 0.3058 - output_2_loss: 0.4021
Epoch 3/3
6821/6821 [==============================] - 3193s 468ms/step - loss: 0.5367 - output_1_loss: 0.2251 - output_2_loss: 0.3116


# 3) 테스트 데이터에 대한 예측


In [28]:
def predict_test_data_by_idx(idx):
    context = tokenizer.decode(X_test[0][idx]).split('[SEP]')[0]
    question = tokenizer.decode(X_test[0][idx]).split('[SEP]')[1]

    print("본문:", context)
    print("질문:", question)
    answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
    print('정답:', tokenizer.decode(answer_encoded))

    output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
    start = tf.math.argmax(tf.squeeze(output[0]))
    end = tf.math.argmax(tf.squeeze(output[1]))

    answer_encoded = X_test[0][idx][start:end]
    print("예측:", tokenizer.decode(answer_encoded))
    print('-' * 50)


In [29]:
for i in range(0,10):
    predict_test_data_by_idx(i)

본문: [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 
질문:  임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답: 1989년 2월 15일
예측: 1989년 2월 15
--------------------------------------------------
본문: [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견,

# 4) 더 긴 입력을 위해 모델 바꾸기

BERT 최대길이 512로 하면 512 이상의 문서는 짤림

KoBigBird 모델은 4096 길이까지 받아 KorQuad 본문이 짤리지 않음

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("monologg/kobigbird-bert-base")

train_encodings = tokenizer(train_contexts, train_questions, truncation = True, padding = True)
val_encodings = tokenizer(val_contexts, val_questions, truncation = True, padding = True)


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [ ]:
model = TFBertForQuestionAnswering('monologg/kobigbird-bert-base')
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
model.compile(optimizer = optimizer, loss = [loss, loss])

In [3]:
import nbformat

file_path = "/content/18_5_KoBERT를_이용한_기계_독해 (1).ipynb"
# 1. 노트북 로드
with open(file_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# 2. widgets 메타데이터 제거
if "widgets" in nb.metadata:
    print("Removing 'metadata.widgets'...")
    del nb.metadata["widgets"]

# 3. 다시 저장
with open(file_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ 위젯 메타데이터 제거 완료. GitHub에서 렌더링 가능!")


Removing 'metadata.widgets'...
✅ 위젯 메타데이터 제거 완료. GitHub에서 렌더링 가능!
